<a href="https://colab.research.google.com/github/NShravanReddy/DeepLearning/blob/main/triton/ReLU_triton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import triton
import triton.language as tl
DEVICE='cuda'

In [16]:
@triton.jit
def t_k(x_ptr,
        y_ptr,
        N0,
        BLOCK_SIZE:tl.constexpr):
  pid=tl.program_id(axis=0)
  block_start= BLOCK_SIZE * pid
  offsets = block_start+ tl.arange(0,BLOCK_SIZE)
  mask= offsets < N0
  x=tl.load(x_ptr+offsets,mask=mask)
  y=tl.maximum(x,0.0)
  tl.store(y_ptr+offsets,y,mask=mask)

In [17]:
def t_r_h(x:torch.Tensor, BLOCK_SIZE : int=32) ->torch.Tensor:
  N0=x.numel()
  y=torch.empty_like(x)
  grid= lambda meta :(triton.cdiv(N0,meta['BLOCK_SIZE']),)
  t_k[grid](x,y,N0,BLOCK_SIZE=BLOCK_SIZE)
  return y

In [20]:
if __name__ == '__main__':
    # Create an example input vector on the GPU.
    N = 1024 * 1024  # For instance, 1 million elements.
    x = torch.randn(N, device='cuda', dtype=torch.float32)

    # Apply ReLU using the Triton kernel.
    y_triton = t_r_h(x)

    # Apply ReLU using PyTorch for validation.
    y_torch = torch.relu(x)
    print(y_torch)
    print(y_triton)
    print(abs(y_triton-y_torch))

tensor([0.0510, 1.4117, 1.1519,  ..., 0.0000, 0.0833, 0.0000], device='cuda:0')
tensor([0.0510, 1.4117, 1.1519,  ..., 0.0000, 0.0833, 0.0000], device='cuda:0')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
